In [ ]:
!pip install xgboost-ray scikit-learn pandas pyarrow kagglehub "ray[default]" --upgrade ray

In [14]:
import pandas as pd
import ray
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import time
import glob
import kagglehub
from ray.train.xgboost import XGBoostTrainer
from ray.train import ScalingConfig
from ray.data import from_pandas

In [3]:
path = kagglehub.dataset_download("elemento/nyc-yellow-taxi-trip-data")
print("Path to dataset files:", path)

100%|██████████| 1.78G/1.78G [00:07<00:00, 260MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/elemento/nyc-yellow-taxi-trip-data/versions/2


In [8]:
# Initialize Ray
ray.init(ignore_reinit_error=True, num_cpus=16)

2025-03-02 12:53:36,006	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.11.11
Ray version:,2.43.0
Dashboard:,http://127.0.0.1:8265


In [4]:
local_path = "/root/.cache/kagglehub/datasets/elemento/nyc-yellow-taxi-trip-data/versions/2/*.csv"
csv_files = glob.glob(local_path)
df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)


#df = df.sample(frac=0.5, random_state=42)
#ds = ray.data.read_csv(csv_files)
#ds = ds.limit(int(ds.count() * 0.01))

2025-03-02 12:46:28,864	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


In [16]:
ray.init(ignore_reinit_error=True, num_cpus=32)

2025-02-21 20:21:17,958	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.11.11
Ray version:,2.42.1
Dashboard:,http://127.0.0.1:8265


In [7]:
ray.shutdown()

In [9]:
@ray.remote
def process_data(df: pd.DataFrame):
    # 1 Filter invalid trip_distance:
    df = df[(df["trip_distance"] != 0) & (df["trip_distance"].notna())]

    df = df[(df["tpep_pickup_datetime"] != 0) & (df["tpep_pickup_datetime"].notna())]

    df = df[(df["tpep_dropoff_datetime"] != 0) & (df["tpep_dropoff_datetime"].notna())]

    # 2 Outlier Handling (Quantile-based):
    trip_distance_quantiles = df["trip_distance"].quantile([0.01, 0.99])
    fare_amount_quantiles = df["fare_amount"].quantile([0.01, 0.99])
    df = df[
        (df["trip_distance"] >= trip_distance_quantiles[0.01])
        & (df["trip_distance"] <= trip_distance_quantiles[0.99])
        & (df["fare_amount"] >= fare_amount_quantiles[0.01])
        & (df["fare_amount"] <= fare_amount_quantiles[0.99])
    ]

    # 3 Datetime Features:
    df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"], errors='coerce')
    df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"], errors='coerce')

    # Access datetime properties directly without 'dt' and assign to new columns
    df[['pickup_hour','day_of_week']] = pd.DataFrame(df['tpep_pickup_datetime'].apply(lambda x: [x.hour, x.dayofweek]).tolist(), index= df.index)
    df.loc[:, "is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)

    # 4. Trip Duration:
    df.loc[:, "trip_duration"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60.0

    # 5 Filter for Valid Fare Amounts:
    df = df[df["fare_amount"] > 0]

    df = df.dropna(subset=["pickup_hour", "is_weekend", "trip_duration", "fare_amount" , "trip_distance"])


    return df

In [49]:
#processed_ds = ds.map_batches(process_data, batch_format="pandas")
#num_rows = processed_ds.count()
#print(f"Number of rows: {num_rows}")
#results = ray.get(process_data.remote(df))

2025-02-21 18:50:18,164	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-02-21_18-50-09_431668_10449/logs/ray-data
2025-02-21 18:50:18,165	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV] -> TaskPoolMapOperator[MapBatches(process_data)] -> AggregateNumRows[AggregateNumRows]


Running 0: 0.00 row [00:00, ? row/s]

- ReadCSV->SplitBlocks(4) 1: 0.00 row [00:00, ? row/s]

- MapBatches(process_data) 2: 0.00 row [00:00, ? row/s]

- AggregateNumRows 3: 0.00 row [00:00, ? row/s]

Number of rows: 45833988


In [10]:
#num_chunks = 4
#chunk_size = len(df) // num_chunks

# Create chunks in one step
#chunks = [df[i * chunk_size : (i + 1) * chunk_size] for i in range(num_chunks - 1)]
#chunks.append(df[(num_chunks - 1) * chunk_size:])

#results = ray.get([process_data.remote(chunk) for chunk in chunks])
# Combine the processed chunks into a single DataFrame
#processed_df = pd.concat(results, ignore_index=True)


results = ray.get(process_data.remote(df))
processed_df = results

# Show the first 5 rows of the processed dataset
print(processed_df.head(5))

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2015-01-15 19:05:39   2015-01-15 19:23:42                1   
1         1  2015-01-10 20:33:38   2015-01-10 20:53:28                1   
2         1  2015-01-10 20:33:38   2015-01-10 20:43:41                1   
3         1  2015-01-10 20:33:39   2015-01-10 20:35:31                1   
4         1  2015-01-10 20:33:39   2015-01-10 20:52:58                1   

   trip_distance  pickup_longitude  pickup_latitude  RateCodeID  \
0           1.59        -73.993896        40.750111         1.0   
1           3.30        -74.001648        40.724243         1.0   
2           1.80        -73.963341        40.802788         1.0   
3           0.50        -74.009087        40.713818         1.0   
4           3.00        -73.971176        40.762428         1.0   

  store_and_fwd_flag  dropoff_longitude  ...  mta_tax  tip_amount  \
0                  N         -73.974785  ...      0.5        3.25   
1       

In [ ]:
#feature_columns = ["pickup_hour", "passenger_count", "is_weekend", "trip_duration" , "trip_distance"]
#target_column = "fare_amount"

#train_dataset = processed_ds.map_batches(
#    lambda df: pd.concat([pd.DataFrame(df[feature_columns], columns=feature_columns), df[target_column]], axis=1),
#    batch_format="pandas"
#)
#test_dataset = processed_ds.map_batches(
#    lambda df: pd.concat([pd.DataFrame(df[feature_columns], columns=feature_columns), df[target_column]], axis=1),
#    batch_format="pandas"
#)

In [11]:
feature_columns = ["pickup_hour", "passenger_count",
                   "is_weekend", "trip_duration", "trip_distance"]
target_column = "fare_amount"


X_train, X_test, y_train, y_test = train_test_split(
    processed_df[feature_columns], processed_df[target_column], test_size=0.2, random_state=42
)


In [12]:
X_train.head()

,pickup_hour,passenger_count,is_weekend,trip_duration,trip_distance
2445482,2,3,1,30.083333,5.59
20808674,17,1,0,32.533333,6.50
904211,17,5,0,13.450000,2.67
14641190,19,1,0,21.616667,3.80
43236269,20,1,0,14.616667,2.70


In [15]:
train_dataset = from_pandas(pd.concat([pd.DataFrame(X_train, columns=feature_columns), y_train], axis=1))
test_dataset = from_pandas(pd.concat([pd.DataFrame(X_test, columns=feature_columns), y_test], axis=1))

In [16]:
train_dataset.show(5)

2025-03-02 12:59:50,097	INFO dataset.py:2787 -- Tip: Use `take_batch()` instead of `take() / show()` to return records in pandas or numpy batch format.
2025-03-02 12:59:50,103	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-03-02_12-53-34_086702_3020/logs/ray-data
2025-03-02 12:59:50,104	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> LimitOperator[limit=5]


Running 0: 0.00 row [00:00, ? row/s]

- limit=5 1: 0.00 row [00:00, ? row/s]

{'pickup_hour': 2, 'passenger_count': 3, 'is_weekend': 1, 'trip_duration': 30.083333333333332, 'trip_distance': 5.59, 'fare_amount': 22.5}
{'pickup_hour': 17, 'passenger_count': 1, 'is_weekend': 0, 'trip_duration': 32.53333333333333, 'trip_distance': 6.5, 'fare_amount': 25.0}
{'pickup_hour': 17, 'passenger_count': 5, 'is_weekend': 0, 'trip_duration': 13.45, 'trip_distance': 2.67, 'fare_amount': 12.0}
{'pickup_hour': 19, 'passenger_count': 1, 'is_weekend': 0, 'trip_duration': 21.616666666666667, 'trip_distance': 3.8, 'fare_amount': 16.5}
{'pickup_hour': 20, 'passenger_count': 1, 'is_weekend': 0, 'trip_duration': 14.616666666666667, 'trip_distance': 2.7, 'fare_amount': 12.5}


In [18]:
xgboost_config = {
    "objective": "reg:squarederror",
    "max_depth": 6,
    "eta": 0.1,
    "subsample": 0.8,
    "eval_metric": ["rmse", "mae"],
}

trainer = XGBoostTrainer(
    datasets={"train": train_dataset, "test": test_dataset},
    label_column=target_column,
    params=xgboost_config,
    num_boost_round=50,
    scaling_config=ScalingConfig(num_workers=16)
)

In [19]:
result = trainer.fit()

2025-03-02 13:01:01,862	INFO tensorboardx.py:193 -- pip install "ray[tune]" to see TensorBoard files.
2025-03-02 13:01:01,862	WARNING callback.py:136 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`



View detailed results here: /root/ray_results/XGBoostTrainer_2025-03-02_13-01-01

Training started without custom configuration.


(XGBoostTrainer pid=18245) Started distributed worker processes: 
(XGBoostTrainer pid=18245) - (node_id=8e63983b3dfc7afcaa77dcaf79dfbacd83bb24033ba06d9d8da679d1, ip=172.28.0.12, pid=18342) world_rank=0, local_rank=0, node_rank=0
(XGBoostTrainer pid=18245) - (node_id=8e63983b3dfc7afcaa77dcaf79dfbacd83bb24033ba06d9d8da679d1, ip=172.28.0.12, pid=18343) world_rank=1, local_rank=1, node_rank=0
(XGBoostTrainer pid=18245) - (node_id=8e63983b3dfc7afcaa77dcaf79dfbacd83bb24033ba06d9d8da679d1, ip=172.28.0.12, pid=18344) world_rank=2, local_rank=2, node_rank=0
(XGBoostTrainer pid=18245) - (node_id=8e63983b3dfc7afcaa77dcaf79dfbacd83bb24033ba06d9d8da679d1, ip=172.28.0.12, pid=18348) world_rank=3, local_rank=3, node_rank=0
(XGBoostTrainer pid=18245) - (node_id=8e63983b3dfc7afcaa77dcaf79dfbacd83bb24033ba06d9d8da679d1, ip=172.28.0.12, pid=18345) world_rank=4, local_rank=4, node_rank=0
(XGBoostTrainer pid=18245) - (node_id=8e63983b3dfc7afcaa77dcaf79dfbacd83bb24033ba06d9d8da679d1, ip=172.28.0.12, pid=183

(pid=19678) Running 0: 0.00 row [00:00, ? row/s]

(pid=19678) - split(16, equal=True) 1: 0.00 row [00:00, ? row/s]

...


Training finished iteration 49 at 2025-03-02 13:02:23. Total running time: 1min 21s
+-------------------------------+
| Training result               |
+-------------------------------+
| checkpoint_dir_name           |
| time_this_iter_s      0.44018 |
| time_total_s          79.3901 |
| training_iteration         49 |
| test-mae              0.33449 |
| test-rmse             1.16699 |
| train-mae             0.33446 |
| train-rmse            1.16914 |
+-------------------------------+


(XGBoostTrainer pid=18245) [13:02:23] [48]	train-rmse:1.16914	train-mae:0.33446	test-rmse:1.16699	test-mae:0.33449



Training finished iteration 50 at 2025-03-02 13:02:24. Total running time: 1min 22s
+-------------------------------+
| Training result               |
+-------------------------------+
| checkpoint_dir_name           |
| time_this_iter_s      0.41405 |
| time_total_s          79.8042 |
| training_iteration         50 |
| test-mae              0.33377 |
| test-rmse             1.16643 |
| train-mae             0.33374 |
| train-rmse            1.16855 |
+-------------------------------+

Training finished iteration 51 at 2025-03-02 13:02:24. Total running time: 1min 22s
+-----------------------------------------+
| Training result                         |
+-----------------------------------------+
| checkpoint_dir_name   checkpoint_000000 |
| time_this_iter_s                0.00728 |
| time_total_s                   79.81144 |
| training_iteration                   51 |
| test-mae                        0.33377 |
| test-rmse                       1.16643 |
| train-mae               

(XGBoostTrainer pid=18245) [13:02:24] [49]	train-rmse:1.16855	train-mae:0.33374	test-rmse:1.16643	test-mae:0.33377
(RayTrainWorker pid=18342) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/XGBoostTrainer_2025-03-02_13-01-01/XGBoostTrainer_649e6_00000_0_2025-03-02_13-01-01/checkpoint_000000)
2025-03-02 13:02:25,474	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/XGBoostTrainer_2025-03-02_13-01-01' in 0.0056s.



Training completed after 51 iterations at 2025-03-02 13:02:25. Total running time: 1min 23s

